In [16]:
import shapefile
import pandas as pd
from pyproj import Proj, Transformer

In [17]:
node_path = './[2023-03-29]NODELINKDATA (1)/MOCT_NODE.shp'
link_path = './[2023-03-29]NODELINKDATA (1)/MOCT_LINK.shp'
node_reader = shapefile.Reader(node_path, encoding='cp949')
link_reader = shapefile.Reader(link_path, encoding='cp949')

In [18]:
fields_node = [x[0] for x in node_reader.fields][1:]
records_node = node_reader.records()
shapes = [s.points for s in node_reader.shapes()]

In [19]:
fields_link = [x[0] for x in link_reader.fields][1:]
records_link = link_reader.records()

In [20]:
node_dataframe = pd.DataFrame(columns=fields_node, data=records_node)
node_dataframe = node_dataframe.assign(coords=shapes)

In [21]:
node_dataframe

,NODE_ID,NODE_TYPE,NODE_NAME,TURN_P,REMARK,coords
0,1240094800,102,고덕동467-4,0,,"[[213865.94130000006, 551566.1490000002]]"
1,1240094900,101,고덕동465-7,0,,"[[213867.16189999972, 551673.7378000002]]"
2,1240095000,102,고덕동468,0,,"[[213974.7910000002, 551570.2592999991]]"
3,1240095100,101,고덕동353-27,0,,"[[213849.36390000023, 551770.9449000005]]"
4,1770084301,102,오꼬꼬,0,,"[[184456.06840000022, 276127.4079]]"
...,...,...,...,...,...,...
201748,2440328800,106,기흥IC,0,,"[[209054.9463359689, 513756.7071187099]]"
201749,2440328900,106,기흥IC,0,,"[[208870.58500115495, 513801.9651721073]]"
201750,2440329100,101,기흥IC,0,,"[[209010.29053902486, 513607.8283160362]]"
201751,2440042400,106,기흥TG앞,0,,"[[209222.41526748805, 513402.9632708272]]"


In [22]:
link_dataframe = pd.DataFrame(columns=fields_link, data=records_link)

In [23]:
link_dataframe

,LINK_ID,F_NODE,T_NODE,LANES,ROAD_RANK,ROAD_TYPE,ROAD_NO,ROAD_NAME,ROAD_USE,MULTI_LINK,CONNECT,MAX_SPD,REST_VEH,REST_W,REST_H,LENGTH,REMARK
0,2560030400,2560013900,2560014500,2,103,000,7,동해대로,0,0,000,80,5,0,0,901.589333,
1,2560038800,2560018000,2560018100,2,107,000,-,오십천로,0,0,000,50,0,0,0,118.955354,
2,2560038900,2560017900,2560018400,2,103,000,7,동해대로,0,0,000,50,0,0,0,123.251630,
3,2560039000,2560018400,2560017900,2,103,000,7,동해대로,0,0,000,50,0,0,0,123.623122,
4,2560039100,2560018200,2560018500,3,107,000,-,중앙로,0,0,000,50,0,0,0,128.922824,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
542710,2560028904,2560013102,2560013101,1,105,000,28,강원남부로,0,0,000,60,0,0,0,979.523801,
542711,2560035200,2560016600,2560016400,3,107,000,-,중앙로,0,0,000,50,0,0,0,258.670733,
542712,2020024500,2020009800,2020009200,4,107,000,-,덕영대로,0,0,000,60,0,0,0,101.933283,
542713,2020036000,2020013200,2020013000,1,107,000,-,수성로157번길,0,0,000,50,0,0,0,40.006728,


In [24]:
for idx, row in node_dataframe.iterrows():
    if isinstance(row['NODE_NAME'], bytes):
        row['NODE_NAME'] = row['NODE_NAME'].decode('utf-8') 

In [25]:
transformer = Transformer.from_crs("EPSG:5186", "EPSG:4326")
# transformer.transform(12, 12)
latitude = []
longitude = []
for idx, row in node_dataframe.iterrows():
    x, y = row.coords[0][0], row.coords[0][1]
    nx, ny = transformer.transform(x, y)
    latitude.append(ny)
    longitude.append(nx)
node_dataframe['latitude'] = latitude
node_dataframe['longitude'] = longitude
del node_dataframe['coords']

In [26]:
node_dataframe

,NODE_ID,NODE_TYPE,NODE_NAME,TURN_P,REMARK,latitude,longitude
0,1240094800,102,고덕동467-4,0,,130.825185,34.460270
1,1240094900,101,고덕동465-7,0,,130.826354,34.460245
2,1240095000,102,고덕동468,0,,130.825275,34.461248
3,1240095100,101,고덕동353-27,0,,130.827402,34.460052
4,1770084301,102,오꼬꼬,0,,127.826466,34.252276
...,...,...,...,...,...,...,...
201748,2440328800,106,기흥IC,0,,130.412723,34.429156
201749,2440328900,106,기흥IC,0,,130.413147,34.427484
201750,2440329100,101,기흥IC,0,,130.411090,34.428800
201751,2440042400,106,기흥TG앞,0,,130.408942,34.430771


In [28]:
link_dataframe

,LINK_ID,F_NODE,T_NODE,LANES,ROAD_RANK,ROAD_TYPE,ROAD_NO,ROAD_NAME,ROAD_USE,MULTI_LINK,CONNECT,MAX_SPD,REST_VEH,REST_W,REST_H,LENGTH,REMARK
0,2560030400,2560013900,2560014500,2,103,000,7,동해대로,0,0,000,80,5,0,0,901.589333,
1,2560038800,2560018000,2560018100,2,107,000,-,오십천로,0,0,000,50,0,0,0,118.955354,
2,2560038900,2560017900,2560018400,2,103,000,7,동해대로,0,0,000,50,0,0,0,123.251630,
3,2560039000,2560018400,2560017900,2,103,000,7,동해대로,0,0,000,50,0,0,0,123.623122,
4,2560039100,2560018200,2560018500,3,107,000,-,중앙로,0,0,000,50,0,0,0,128.922824,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
542710,2560028904,2560013102,2560013101,1,105,000,28,강원남부로,0,0,000,60,0,0,0,979.523801,
542711,2560035200,2560016600,2560016400,3,107,000,-,중앙로,0,0,000,50,0,0,0,258.670733,
542712,2020024500,2020009800,2020009200,4,107,000,-,덕영대로,0,0,000,60,0,0,0,101.933283,
542713,2020036000,2020013200,2020013000,1,107,000,-,수성로157번길,0,0,000,50,0,0,0,40.006728,


In [29]:
node_dataframe.to_csv('node.csv')
link_dataframe.to_csv('link.csv')